In [1]:
#https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size=64
epochs=10
latent_dim=256 #latent dimensionalty of encoding space
num_samples=1000 #number of samples to train on
data_path='dataset/fra-eng/fra.txt'

In [6]:
t,s,u="my\tname is daniyal khan\tand i am hero".split('\t')
print(t)

my


In [7]:
#Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r', encoding= 'utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[ : min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t'+ target_text +'\n'
    input_texts.append(input_text)
    target_texts.append (target_text) 
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
print(input_texts,target_texts)
print(input_characters,target_characters)

['Go.', 'Go.', 'Go.', 'Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Who?', 'Wow!', 'Wow!', 'Wow!', 'Duck!', 'Duck!', 'Duck!', 'Fire!', 'Help!', 'Hide.', 'Hide.', 'Jump!', 'Jump.', 'Stop!', 'Stop!', 'Stop!', 'Wait!', 'Wait!', 'Wait!', 'Wait.', 'Wait.', 'Wait.', 'Wait.', 'Begin.', 'Begin.', 'Go on.', 'Go on.', 'Go on.', 'Hello!', 'Hello!', 'Hello.', 'Hello.', 'Hello.', 'Hello.', 'I see.', 'I see.', 'I try.', 'I won!', 'I won!', 'I won.', 'Oh no!', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Smile.', 'Smile.', 'Smile.', 'Sorry?', 'Attack!', 'Attack!', 'Attack!', 'Attack!', 'Buy it.', 'Buy it.', 'Buy it.', 'Buy it.', 'Cheers!', 'Cheers!', 'Cheers!', 'Cheers!', 'Eat it.', 'Eat it.', 'Exhale.', 'Get up.', 'Get up.', 'Get up.', 'Go now.', 'Go now.', 'Go now.', 'Got it!', 'Got it!', 'Got it!', 'Got it?', 'Got it?', 

In [9]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
print('Number of samples:',len(input_texts))
print('Number of unique input tokens',num_encoder_tokens)
print('Number of unique output tokens',num_decoder_tokens)
print('Max sequence length For inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of unique input tokens 56
Number of unique output tokens 76
Max sequence length For inputs: 10
Max sequence length for outputs: 33


In [11]:
#based on sorted list we give index number to each character
input_token_index = dict([(char, i) for i,char in enumerate(input_characters)])
target_token_index =dict([(char, i) for i,char in enumerate(target_characters)])

In [12]:
print(input_token_index)

{' ': 0, '!': 1, "'": 2, ',': 3, '.': 4, '1': 5, '9': 6, '?': 7, 'A': 8, 'B': 9, 'C': 10, 'D': 11, 'E': 12, 'F': 13, 'G': 14, 'H': 15, 'I': 16, 'J': 17, 'K': 18, 'L': 19, 'M': 20, 'N': 21, 'O': 22, 'P': 23, 'R': 24, 'S': 25, 'T': 26, 'U': 27, 'W': 28, 'Y': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [13]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens ), dtype='float32')
decoder_input_data = np.zeros((len (input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32' )
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [17]:
#doing one hot encoder

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t>0 :
        # decoder_target_data will be ahead by one timestep
        # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [18]:
decoder_input_data.shape

(1000, 33, 76)

In [20]:
decoder_target_data.shape

(1000, 33, 76)

In [22]:
#Defining an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True) #latent dim is no. of timestamps we want to consider
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard encoder_outputs and only keep the states. because we dont want output at every state
encoder_states = [state_h, state_c]

In [25]:
#Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_= decoder_lstm(decoder_inputs,initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [28]:
# Define the model that wilL turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Run training
model.compile(optimizer='rmsprop', loss= 'categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size, epochs=epochs,validation_split=0.2)

Epoch 1/10
13/13 [==============================] - 2s 66ms/step - loss: 36.2223 - accuracy: 0.2456 - val_loss: 18.4064 - val_accuracy: 0.4291
Epoch 2/10
13/13 [==============================] - 1s 55ms/step - loss: 37.0077 - accuracy: 0.1914 - val_loss: 18.9568 - val_accuracy: 0.4291
Epoch 3/10
13/13 [==============================] - 1s 56ms/step - loss: 37.7393 - accuracy: 0.1914 - val_loss: 19.3869 - val_accuracy: 0.4291
Epoch 4/10
13/13 [==============================] - 1s 55ms/step - loss: 38.3624 - accuracy: 0.1936 - val_loss: 19.8527 - val_accuracy: 0.4292
Epoch 5/10
13/13 [==============================] - 1s 56ms/step - loss: 39.0899 - accuracy: 0.1973 - val_loss: 20.3438 - val_accuracy: 0.4294
Epoch 6/10
13/13 [==============================] - 1s 55ms/step - loss: 39.7495 - accuracy: 0.1974 - val_loss: 20.8171 - val_accuracy: 0.4292
Epoch 7/10
13/13 [==============================] - 1s 57ms/step - loss: 40.5260 - accuracy: 0.1976 - val_loss: 21.2588 - val_accuracy: 0.4291

In [31]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

for seq_index in range(100):
# Take one sequence (part of the training set)
# for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence (input_seq)
    print ('----------------------------------------')
    print ('Input sentence:',input_texts[seq_index])
    print('Decoded sentence:',decoded_sentence)

1/1 [==============================] - 0s 110ms/step
----------------------------------------
Input sentence: Go.
Decoded sentence: 

1/1 [==============================] - 0s 6ms/step
----------------------------------------
Input sentence: Go.
Decoded sentence: 

1/1 [==============================] - 0s 6ms/step
----------------------------------------
Input sentence: Go.
Decoded sentence: 

1/1 [==============================] - 0s 7ms/step
----------------------------------------
Input sentence: Go.
Decoded sentence: 

1/1 [==============================] - 0s 7ms/step
----------------------------------------
Input sentence: Hi.
Decoded sentence: 

1/1 [==============================] - 0s 6ms/step
----------------------------------------
Input sentence: Hi.
Decoded sentence: 

1/1 [==============================] - 0s 6ms/step
----------------------------------------
Input sentence: Run!
Decoded sentence: 

1/1 [==============================] - 0s 7ms/step
----------------------